# Question 1: 

In [1]:
import torch
import torch.nn as nn
import numpy as np
import math
from sklearn.metrics import mean_squared_error
import torchvision

In [2]:
input=torch.randn(1,3,32,32)

## 1-1 : torch.nn.MaxPool2d

> torch_output

In [3]:

m=nn.MaxPool2d(kernel_size=2, stride=1, padding=0,dilation=1, return_indices=False, ceil_mode=False)
torch_out=m(input)

In [4]:
torch_out.size()

torch.Size([1, 3, 31, 31])

In [5]:
class myMaxPool2d(nn.Module):
    def __init__(self,kernel_size=2, stride=1, padding=0,dilation=1, return_indices=False, ceil_mode=False):
        super(myMaxPool2d,self).__init__()
        self.kernel_size=kernel_size
        self.stride=stride
        self.padding=padding
        self.dilation=dilation
        
    def forward(self,input):
        kernel_width=self.kernel_size
        kernel_height=self.kernel_size
               
        padding_width = self.padding
        padding_height = self.padding
        
        stride_width = self.stride
        stride_height = self.stride
        
        dilation_width = self.dilation
        dilation_height = self.dilation
        
        x_batch=input.shape[0]
        x_channel=input.shape[1]
        x_height=input.shape[2]
        x_weight=input.shape[3]
        
        y_width = math.floor((x_weight+padding_width*2-dilation_width*(kernel_width-1)-1)/stride_width+1)
        y_height = math.floor((x_height+padding_height*2-dilation_height*(kernel_height-1)-1)/stride_width+1)
       
        #print(x_weight+padding_weight*2)
        #print((-dilation_weight*(kernel_weight-1)-1)/stride_weight)
              
        #print("y_weight=%d,y_height=%d"%(y_weight,y_height))
       
        y=np.zeros((x_batch,x_channel,y_height,y_width))

        for i in range(x_batch):
            for j in range(x_channel):
                for k in range(y_height):
                    for l in range(y_width): 
                        ll = l*stride_width
                        lr = ll + kernel_width
                        kl = k*stride_height
                        kr = kl+kernel_height
                        #print("i=%d j=%d l=%d:%d r=%d:%d"%(i,j,ll,lr,kl,kr))
                        y[i,j,k,l]=np.amax(input[i,j,kl:kr,ll:lr])
        return(y)

>  my_output

In [6]:
impl=myMaxPool2d()
my_out=impl(input.numpy())
#print(my_output.shape)

> Check the difference

In [7]:
mean_squared_error(torch_out.numpy().flatten(),my_out.flatten())

0.0

***

## 1-2: AvgPool2d

> torch output

In [8]:
m=nn.AvgPool2d(kernel_size=2, stride=1, padding=0,ceil_mode=False, count_include_pad=True,divisor_override=None)
torch_out=m(input)

In [9]:
class myAvgPool2d(nn.Module):
    def __init__(self,kernel_size=2, stride=1, padding=0,ceil_mode=False, count_include_pad=True,divisor_override=None):
        super(myAvgPool2d,self).__init__()
        self.kernel_size=kernel_size
        self.stride=stride
        self.padding=padding
        
    def forward(self,input):
        kernel_width=self.kernel_size
        kernel_height=self.kernel_size
               
        padding_width = self.padding
        padding_height = self.padding
        
        stride_width = self.stride
        stride_height = self.stride
        
        
        x_batch=input.shape[0]
        x_channel=input.shape[1]
        x_height=input.shape[2]
        x_width=input.shape[3]
        
        #print("%x_batch=d, x_channel=%d, x_weight=%d,x_height=%d"%(x_batch,x_channel,x_weight,x_height))
       
        # for simplicity, assume 
        # (a) ceiling =FALSE; 
        # (b) square kernel and squre padding
        
        y_width = math.floor((x_width+padding_width*2-(kernel_width-1)-1)/stride_width+1)
        y_height = math.floor((x_height+padding_height*2-(kernel_height-1)-1)/stride_height+1)
       
        #print("y_weight=%d,y_height=%d"%(y_weight,y_height))
       
        y=np.zeros((x_batch,x_channel,y_height,y_width))

        for i in range(x_batch):
            for j in range(x_channel):
                for k in range(y_height):
                    for l in range(y_width): 
                        ll = l*stride_width
                        lr = ll + kernel_width
                        kl = k*stride_height
                        kr = kl+kernel_height
                        #print("i=%d j=%d l=%d:%d r=%d:%d"%(i,j,ll,lr,kl,kr))
                        y[i,j,k,l]=np.average(input[i,j,kl:kr,ll:lr])
        return(y)

> my output

In [10]:
impl=myAvgPool2d()
my_out=impl(input.numpy())

> Check the difference

In [11]:
mean_squared_error(torch_out.numpy().flatten(),my_out.flatten())

0.0

***

## 1-3: Conv2d

> torch output

In [12]:
m=nn.Conv2d(in_channels=3, out_channels=6,kernel_size=3, stride=1, padding=0, dilation=1, groups=1,bias=True, padding_mode='zeros')
torch_out=m(input)
torch_weight = m.weight.data
torch_bias = m.bias.data

In [13]:
print(torch_weight.shape)
print(torch_bias.shape)

torch.Size([6, 3, 3, 3])
torch.Size([6])


In [14]:
class myConv2d(nn.Module):
    def __init__(self,in_channels=3, out_channels=6,kernel_size=3, stride=1, padding=0, dilation=1, groups=1,bias=True, padding_mode='zeros'):
        super(myConv2d,self).__init__()
        self.kernel_size=kernel_size
        self.stride=stride
        self.padding=padding
                 
        self.out_channels=out_channels         
        
    def forward(self,input):
        kernel_width=self.kernel_size
        kernel_height=self.kernel_size
                   
        padding_width = self.padding
        padding_height = self.padding
        
        stride_width = self.stride
        stride_height = self.stride
        
        
        x_batch=input.shape[0]
        x_channel=input.shape[1]
        x_height=input.shape[2]
        x_width=input.shape[3]
        
        #print("%x_batch=d, x_channel=%d, x_weight=%d,x_height=%d"%(x_batch,x_channel,x_weight,x_height))
       
        # for simplicity, assume 
        # (a) ceiling =FALSE; 
        # (b) square kernel and squre padding
        
        y_width = math.floor((x_width+padding_width*2-(kernel_width-1)-1)/stride_width+1)
        y_height = math.floor((x_height+padding_height*2-(kernel_height-1)-1)/stride_height+1)
       
        #print("y_weight=%d,y_height=%d"%(y_weight,y_height))
       
        y=np.zeros((x_batch, self.out_channels,y_height,y_width))

        for i in range(x_batch):
            for c in range(self.out_channels):
                 for j in range(x_channel):
                        for k in range(y_height):
                            for l in range(y_width): 
                                ll = l*stride_width
                                lr = ll + kernel_width
                                kl = k*stride_height
                                kr = kl+kernel_height
                                #print("i=%d j=%d l=%d:%d r=%d:%d"%(i,j,ll,lr,kl,kr))
                                y[i, c, k, l] += (input[i, j, kl:kr,ll:lr] * torch_weight[c][j].numpy()).sum()
        
            for i in range(self.out_channels):
                y[0][i] += torch_bias[i].numpy()
        
        return(y)

> my output

In [15]:
impl=myConv2d()
my_out=impl(input.numpy())

> Check the difference

In [17]:
torch_flatten=torch_out.detach().numpy().flatten()
my_flatten=my_out.flatten()
mean_squared_error(torch_flatten,my_flatten)

4.838458065133246e-15

> some minor difference due to floating point arithmatics

In [18]:
torch_flatten

array([ 0.9718749 ,  0.58248353,  0.7136466 , ...,  0.14838868,
       -0.77434736, -0.26057613], dtype=float32)

In [19]:
my_flatten

array([ 0.97187494,  0.58248357,  0.7136467 , ...,  0.14838862,
       -0.77434734, -0.26057605])

***

# 1.4 Conv2d with dilation>1

> torch output

In [20]:
m=nn.Conv2d(in_channels=3, out_channels=6,kernel_size=5, stride=2, padding=0, dilation=2, groups=1,bias=True, padding_mode='zeros')
torch_out=m(input)
torch_weight = m.weight.data
torch_bias = m.bias.data

print(torch_out.size())
print(torch_weight.size())
print(torch_bias.size())

torch.Size([1, 6, 12, 12])
torch.Size([6, 3, 5, 5])
torch.Size([6])


In [21]:
class myConv2d_2(nn.Module):
    def __init__(self,in_channels=3, out_channels=6,kernel_size=5, stride=2, padding=0, dilation=2, groups=1,bias=True, padding_mode='zeros'):
        super(myConv2d_2,self).__init__()
        self.kernel_size=kernel_size
        self.stride=stride
        self.padding=padding
        
        self.dilation = dilation
                 
        self.out_channels=out_channels         
        
    def forward(self,input):
        kernel_width=self.kernel_size
        kernel_height=self.kernel_size
                   
        padding_width = self.padding
        padding_height = self.padding
        
        stride_width = self.stride
        stride_height = self.stride
        
        
        x_batch=input.shape[0]
        x_channel=input.shape[1]
        x_height=input.shape[2]
        x_width=input.shape[3]
        
        y_width = math.floor((x_width+padding_width*2-self.dilation*(kernel_width-1)-1)/stride_width+1)
        y_height = math.floor((x_height+padding_height*2-self.dilation*(kernel_height-1)-1)/stride_height+1)
    
        #print("y_weight=%d,y_height=%d"%(y_width,y_height))
    
    
        y=np.zeros((x_batch, self.out_channels,y_height,y_width))
        
        weights = np.zeros((self.out_channels,x_channel,kernel_height* self.dilation-1,kernel_width* self.dilation-1))
        
        #print(y.shape)
        #print(weights.shape)
        
        for c in range(self.out_channels):
            for j in range(x_channel):
                for k in range(kernel_height):
                    for l in range(kernel_width):
                        weights[c, j, k * self.dilation, l * self.dilation] = torch_weight[c, j, k, l]

        for i in range(x_batch):
            for c in range(self.out_channels):
                 for j in range(x_channel):
                        for k in range(y_height):
                            for l in range(y_width): 
                                kl = k*stride_height
                                kr = kl+kernel_height*self.dilation-1
                                
                                ll = l*stride_width
                                lr = ll + kernel_width*self.dilation-1
                                #print("i=%d j=%d l=%d:%d r=%d:%d"%(i,j,ll,lr,kl,kr))
                                y[i, c, k, l] += (input[i, j, kl:kr,ll:lr] * weights[c][j]).sum()
        
            for i in range(self.out_channels):
                y[0][i] += torch_bias[i].numpy()
        
        return(y)

> my output

In [22]:
impl=myConv2d_2()
my_out=impl(input.numpy())

> Check the difference

In [25]:
torch_flatten=torch_out.detach().numpy().flatten()
my_flatten=my_out.flatten()
mean_squared_error(torch_flatten,my_flatten)

8.664802881307877e-15

In [26]:
torch_flatten

array([-7.47435331e-01,  1.28319705e+00,  2.86253333e-01,  9.90206957e-01,
        4.18511450e-01,  6.93405271e-01,  3.93564194e-01,  3.99817526e-01,
       -4.40417141e-01,  4.92162667e-02,  1.54106930e-01, -9.42227960e-01,
        6.69521332e-01,  5.27054310e-01,  1.87416387e+00,  8.46657693e-01,
        2.50869244e-01,  2.73079753e-01, -8.36745724e-02, -6.45477712e-01,
       -1.06390572e+00, -1.48029232e+00,  2.06011981e-02,  1.85969427e-01,
       -3.51295263e-01,  2.85909176e-01,  3.57178710e-02,  3.68878245e-01,
       -5.24603069e-01, -1.27037108e+00, -1.15260541e+00,  5.79615496e-02,
       -9.56859410e-01, -1.60745525e+00, -6.18579574e-02,  2.41684064e-01,
       -1.24229980e+00, -5.89672104e-02, -1.07209183e-01, -5.63483417e-01,
       -2.24590927e-01, -7.64131486e-01, -2.31330022e-01,  2.82435656e-01,
       -2.46507287e-01,  9.48250592e-01, -1.40352510e-02,  4.64826941e-01,
       -7.28797138e-01, -1.06858456e+00, -4.26149845e-01, -7.06250608e-01,
       -8.76295388e-01, -

In [27]:
my_flatten

array([-7.47435362e-01,  1.28319701e+00,  2.86253268e-01,  9.90206848e-01,
        4.18511416e-01,  6.93405202e-01,  3.93564197e-01,  3.99817559e-01,
       -4.40417110e-01,  4.92161089e-02,  1.54106860e-01, -9.42227838e-01,
        6.69521307e-01,  5.27054405e-01,  1.87416408e+00,  8.46657665e-01,
        2.50869324e-01,  2.73079746e-01, -8.36744966e-02, -6.45477723e-01,
       -1.06390568e+00, -1.48029211e+00,  2.06012495e-02,  1.85969433e-01,
       -3.51295142e-01,  2.85909229e-01,  3.57179552e-02,  3.68878266e-01,
       -5.24603078e-01, -1.27037115e+00, -1.15260535e+00,  5.79615234e-02,
       -9.56859429e-01, -1.60745550e+00, -6.18579179e-02,  2.41684113e-01,
       -1.24229970e+00, -5.89671105e-02, -1.07209190e-01, -5.63483455e-01,
       -2.24590809e-01, -7.64131532e-01, -2.31329821e-01,  2.82435635e-01,
       -2.46507314e-01,  9.48250580e-01, -1.40350905e-02,  4.64826914e-01,
       -7.28797144e-01, -1.06858463e+00, -4.26149907e-01, -7.06250583e-01,
       -8.76295397e-01, -

***

# 1.5 ConvTranspose2d

> torch output

In [28]:
m=nn.ConvTranspose2d(in_channels=3, out_channels=4,kernel_size=3, stride=1, padding=0, output_padding=0,groups=1, bias=True, dilation=1, padding_mode='zeros')
torch_out=m(input)
torch_weight = m.weight.data
torch_bias = m.bias.data

print(torch_out.size())
print(torch_weight.size())
print(torch_bias.size())

torch.Size([1, 4, 34, 34])
torch.Size([3, 4, 3, 3])
torch.Size([4])


In [29]:
class myConvTranspose2d(nn.Module):
    def __init__(self,in_channels=3, out_channels=4,kernel_size=3, stride=1, padding=0, output_padding=0,groups=1, bias=True, dilation=1, padding_mode='zeros'):
        super(myConvTranspose2d,self).__init__()
        self.kernel_size=kernel_size
        self.stride=stride
        self.padding=padding
        
        self.dilation = dilation
                 
        self.out_channels=out_channels   
        self.output_padding=output_padding
        
    def forward(self,input):
        kernel_width=self.kernel_size
        kernel_height=self.kernel_size
                   
        padding_width = self.padding
        padding_height = self.padding
        
        stride_width = self.stride
        stride_height = self.stride
        
        
        x_batch=input.shape[0]
        x_channel=input.shape[1]
        x_height=input.shape[2]
        x_width=input.shape[3]
        
       
        y_height = (x_height-1)*stride_height -2*padding_width + self.dilation*(kernel_height-1)+self.output_padding+1
        y_width  = (x_width-1)*stride_width -2*padding_height + self.dilation*(kernel_width-1)+self.output_padding+1
        #print("y_weight=%d,y_height=%d"%(y_width,y_height))
    
    
        y=np.zeros((x_batch, self.out_channels,y_height,y_width))
        #print(y.shape)         
        
       
        #print(y.shape)
        #print(weights.shape)

        for i in range(x_batch):
            for c in range(self.out_channels):
                 for j in range(x_channel):
                        for k in range(x_height):
                            for l in range(x_width):
                                y[i, c, k:k+kernel_height, l:l+kernel_width] +=input[i, j, k, l] * torch_weight[j][c].numpy()       
        for i in range(self.out_channels):
            y[0][i] += torch_bias[i].numpy()
        
        return(y)

> my output

In [30]:
impl=myConvTranspose2d()
my_out=impl(input.numpy())

> Check the difference

In [31]:
torch_flatten=torch_out.detach().numpy().flatten()
my_flatten=my_out.flatten()
mean_squared_error(torch_flatten,my_flatten)

1.2756529552626814e-15

***

# 2.1 flatten

> torch output

In [32]:
torch_out=torch.flatten(input, start_dim=0, end_dim=-1)

> my output

In [33]:
def myflatten(input,start_dim=0, end_dim=-1):
    shape=input.shape
    reshape=1
    for i in shape:
        reshape*=i
    return input.reshape(reshape)   

my_out=myflatten(input.numpy())

> Check the difference

In [34]:
mean_squared_error(torch_out.numpy(),my_out)

0.0

# 2.2 sigmoid

> torch output

In [35]:
torch_out=torch.sigmoid(input)

> my output

In [36]:
def mysigmoid(input):
    print(type(input))
    return 1 / (1 + np.exp(-input))
  
my_out = mysigmoid(input.numpy())

<class 'numpy.ndarray'>


> Check the difference

In [37]:
torch_out.shape

torch.Size([1, 3, 32, 32])

In [38]:
mean_squared_error(list(torch_out.numpy().reshape(-1)),list(my_out.reshape(-1)))

3.525193e-16

***

# 2.3 roi_pool

> torch output

In [39]:
rois = torch.Tensor([[0, 0, 0, 1, 1]])

torch_out = torchvision.ops.roi_pool(input, rois, (2, 2))
print(torch_out)

tensor([[[[ 0.0071, -0.3352],
          [-2.1432, -1.7375]],

         [[ 1.5856,  1.3147],
          [-1.8290,  0.2250]],

         [[-1.0791, -1.7317],
          [-0.2466,  0.2755]]]])


> my output

In [40]:
def my_roi_pool(input, rois, output_size):
    
    top_left_h = int(rois[0][1])
    top_left_w = int(rois[0][2])
    bottom_right_h = int(rois[0][3])
    bottom_right_w = int(rois[0][4])
    
    box_width =  bottom_right_w - top_left_w + 1
    box_height = bottom_right_h - top_left_h + 1

    bin_width = box_width / output_size[0]
    bin_height = box_height / output_size[1]
    
    result = []
    for i in range(input.shape[1]): # channel
      lst = []
      for j in range(output_size[0]):
        r = []
        for k in range(output_size[1]):
          bar = input[..., int(top_left_h + bin_height * j):math.ceil(top_left_h + bin_height * (j + 1)), 
                   int(top_left_w + bin_width * k):math.ceil(top_left_w + bin_width * (k + 1))][0][i]
          r.append(bar.max())
        lst.append(r)
      result.append(lst)

    return np.array(result)
  
my_out = my_roi_pool(input.numpy(), rois, (2, 2))

> Check the difference

In [41]:
mean_squared_error(torch_out.squeeze(0).numpy().reshape(-1),my_out.reshape(-1))

0.0

***

# 2.4 batch_norm

> torch output

In [42]:
channel = input.shape[1]
mean = torch.Tensor(channel, )
var = torch.Tensor(channel, )
for i in range(channel):
  mean[i] = input[0][i].mean()
  var[i] = input[0][i].var()

torch_out = torch.nn.functional.batch_norm(input, mean, var, weight=None, bias=None, training=False, momentum=0.1, eps=1e-05)

> my output

In [43]:
def mybatch_norm(input, running_mean, running_var, momentum=0.1, eps=1e-05):
    mean, var = running_mean[:, None, None], running_var[:, None, None]
    return (input - mean) / torch.sqrt(var + eps)
  
my_out = mybatch_norm(input, mean, var)
print(my_out)

tensor([[[[ 0.0386, -0.3048, -1.1308,  ...,  0.2010,  0.2179, -1.5942],
          [-2.1187, -1.7117,  0.0366,  ...,  0.8217, -0.5954, -1.2811],
          [-0.8231,  1.7391, -0.6309,  ..., -0.4031, -0.6579, -0.9246],
          ...,
          [ 0.6479,  1.1194,  0.4647,  ...,  1.3838,  1.1622, -1.1065],
          [ 0.4087,  0.6839,  0.9628,  ..., -0.2584, -2.4264,  1.2343],
          [ 1.3137, -0.4247,  0.5679,  ...,  0.8844,  0.1244, -0.3869]],

         [[ 1.5238,  1.2577,  0.5633,  ...,  0.8821, -0.2939,  0.6716],
          [-1.8314,  0.1869, -1.1942,  ...,  0.1239,  0.4340,  0.8497],
          [-0.5877,  3.1623,  0.5306,  ...,  0.8043, -0.7589, -1.8619],
          ...,
          [-1.4215, -0.4729, -1.4526,  ..., -0.1781, -0.5190,  0.2982],
          [-0.7468, -0.5997,  0.9374,  ..., -0.2137, -0.7832,  1.0746],
          [-0.3984,  1.0294,  0.9003,  ..., -0.0631, -1.2334, -1.1310]],

         [[-1.0961, -1.7538,  0.1551,  ..., -0.5652, -0.9461, -1.4345],
          [-0.2572,  0.2689,  

> Check the difference

In [44]:
mean_squared_error(torch_out.numpy().reshape(-1),my_out.numpy().reshape(-1))

4.7291874e-15

***

# 2.5 cross_entropy

In [45]:
Input= torch.randn(1,3,32,32)
target = torch.randint(3, (1, 32, 32), dtype = torch.int64)


> torch output

In [46]:
tourch_out=nn.functional.cross_entropy(input, target, weight=None,size_average=None, ignore_index=-100, reduce=None,reduction='mean')

In [47]:
tourch_out

tensor(1.4101)

> torch output

In [48]:
def softmax(input):
    result = torch.zeros(input.shape)
    for i in range(len(input)):
      e = torch.exp(input[i])
      s = torch.sum(e, dim = 0)
      soft = e / s
      result[i] = -torch.log(soft)
    return result
    
def loss(input, label):
    ls = torch.zeros([len(input)])
    for i in range(len(input)):
        s = 0
        for j in range(label.shape[1]):
            for k in range(label.shape[2]):
                index = label[i, j, k]
                p = input[i, index, j ,k]
                s += p
        ls[i] = s / (label.shape[1] * label.shape[2])
    return ls

def my_cross_entropy(input, label):
    log_softmax = softmax(input)
    l = loss(log_softmax, label)
    print(l.size())
    return l.mean()

my_out = my_cross_entropy(input, target.numpy())

torch.Size([1])


> difference

In [49]:
error = torch.abs(my_out - tourch_out).sum().data
squared_error = ((my_out - tourch_out)*(my_out - tourch_out)).sum().data

In [50]:
squared_error.numpy()

array(0., dtype=float32)

***

# 2.6 mse_loss

> torch output

In [58]:
y = torch.randn(1, 3, 32, 32)
torch_out = torch.nn.functional.mse_loss(input, y, size_average=None, reduce=None, reduction='mean')
print(torch_out)

tensor(1.9572)


> my output

In [59]:
def my_mse_loss(X, y):
    sqr = (X - y) ** 2
    return sqr.mean()

my_out = my_mse_loss(input, y)
print(my_out)

tensor(1.9572)


> difference

In [68]:
error=torch.abs(torch_out-my_out).sum().data

tensor(0.)